# Business Case 3: Gift-a-Lot Recommender System

# Contents:
  
 * [Modeling](#Modeling)
 * [Recommender System](#RecommenderSystem)
    * [Recommending a product by user](#recommendingproduct)
    * [Recommending an user based on a product](#recommendinguser)
    * [Recommending products that could be bought together](#productsboughttogether)

## Imports

In [1]:
#! pip install lightfm

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import ceil
import scipy.stats as stats


import plotly.express as px
import plotly.graph_objects as go
import plotly


from scipy import sparse
import scipy.sparse as sp
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score
from sklearn.metrics.pairwise import cosine_similarity

import warnings

C:\Users\andre\anaconda3\envs\dsaadm\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


# Modeling  <a name="Modeling"></a>
>In this step, we decided to implement algorithms to build a Recommender System.

In [3]:
df = pd.read_csv('clean_df.csv') 
df.drop('Unnamed: 0', axis =1, inplace = True)

In [4]:
df.head()

,StockCode,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Month_name,Week,Month_Year,Year_Month,Hour,Day,Date,Value
0,SC85123A,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,December,48,December_2010,2010-12,8,1,2010-12-01 00:00:00,15.30
1,SC71053,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,December,48,December_2010,2010-12,8,1,2010-12-01 00:00:00,20.34
2,SC84406B,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,December,48,December_2010,2010-12,8,1,2010-12-01 00:00:00,22.00
3,SC84029G,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,December,48,December_2010,2010-12,8,1,2010-12-01 00:00:00,20.34
4,SC84029E,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,December,48,December_2010,2010-12,8,1,2010-12-01 00:00:00,20.34


# Recommender System  <a name="RecommenderSystem"></a>
> To implement a Recommender System, we used the Matrix Factorization Model, suggesting products by user, suggesting users that might be interested in a particular product and suggesting products that could be bought together with a specific product. 

## Collaborative Filtering (Customer - Product)

In [5]:
interactions = pd.DataFrame(df.groupby(['CustomerID', 'Description', 'StockCode'])['Quantity'].sum())
interactions.reset_index(inplace = True)

In [6]:
interactions

,CustomerID,Description,StockCode,Quantity
0,12347.0,3D DOG PICTURE PLAYING CARDS,SC84558A,126
1,12347.0,3D SHEET OF CAT STICKERS,SC84559B,12
2,12347.0,3D SHEET OF DOG STICKERS,SC84559A,12
3,12347.0,60 TEATIME FAIRY CAKE CASES,SC84991,72
4,12347.0,72 SWEETHEART FAIRY CAKE CASES,SC84992,72
...,...,...,...,...
267757,18287.0,STAR T-LIGHT HOLDER WILLIE WINKIE,SC23274,12
267758,18287.0,STRAWBERRY BATH SPONGE,SC20961,30
267759,18287.0,STRAWBERRY CERAMIC TRINKET BOX,SC21232,12
267760,18287.0,SWISS CHALET TREE DECORATION,SC21014,48


In [7]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing the quantity purchased
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [8]:
#Creating an interaction matrix, encoding 0 (customer didn't purchased that product) and 1 (he/she purchased the product)

interactions1 = create_interaction_matrix(df = interactions,
                                         user_col = 'CustomerID',
                                         item_col = 'Description',
                                         rating_col = 'Quantity',
                                         norm = True,
                                         threshold = 0)
interactions1.head()

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12352.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Checking the encoding step
interactions1.loc[12347.0].value_counts()

0    3766
1     103
Name: 12347.0, dtype: int64

In [10]:
def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict
    
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [11]:
# Create Customer Dict
user_dict = create_user_dict(interactions=interactions1)
# Create Product dict
products_dict = create_item_dict(df = df,
                               id_col = 'Description',
                               name_col = 'Description')

### Train Test Split

In [12]:
#Transforming the interaction matrix into a sparse matrix 

sparse_matrix = sparse.csr_matrix(interactions1.values)

In [13]:
def _shuffle(uids, iids, data, random_state):

    shuffle_indices = np.arange(len(uids))
    random_state.shuffle(shuffle_indices)

    return (uids[shuffle_indices], iids[shuffle_indices], data[shuffle_indices])

def random_train_test_split(interactions, test_percentage=0.2, random_state=None):
    """
    Randomly split interactions between training and testing.
    This function takes an interaction set and splits it into
    two disjoint sets, a training set and a test set. Note that
    no effort is made to make sure that all items and users with
    interactions in the test set also have interactions in the
    training set; this may lead to a partial cold-start problem
    in the test set.
    To split a sample_weight matrix along the same lines, pass it
    into this function with the same random_state seed as was used
    for splitting the interactions.
    Parameters
    ----------
    interactions: a scipy sparse matrix containing interactions
        The interactions to split.
    test_percentage: float, optional
        The fraction of interactions to place in the test set.
    random_state: int or numpy.random.RandomState, optional
        Random seed used to initialize the numpy.random.RandomState number generator.
        Accepts an instance of numpy.random.RandomState for backwards compatibility.
    Returns
    -------
    (train, test): (scipy.sparse.COOMatrix,
                    scipy.sparse.COOMatrix)
         A tuple of (train data, test data)
    """

    if not sp.issparse(interactions):
        raise ValueError("Interactions must be a scipy.sparse matrix.")

    if not isinstance(random_state, np.random.RandomState):
        random_state = np.random.RandomState(seed=random_state)

    interactions = interactions.tocoo()

    shape = interactions.shape
    uids, iids, data = (interactions.row, interactions.col, interactions.data)

    uids, iids, data = _shuffle(uids, iids, data, random_state)

    cutoff = int((1.0 - test_percentage) * len(uids))

    train_idx = slice(None, cutoff)
    test_idx = slice(cutoff, None)

    train = sp.coo_matrix(
        (data[train_idx], (uids[train_idx], iids[train_idx])),
        shape=shape,
        dtype=interactions.dtype,
    )
    test = sp.coo_matrix(
        (data[test_idx], (uids[test_idx], iids[test_idx])),
        shape=shape,
        dtype=interactions.dtype,
    )

    return train, test

In [14]:
train, test = random_train_test_split(sparse_matrix, test_percentage=0.2, random_state=None)

In [15]:
print("Size (train dataset): ", + train.size) 
print("Size (test dataset): ", + test.size)

Size (train dataset):  214119
Size (test dataset):  53530


In [16]:
model = LightFM(no_components= 30, loss='warp',k=10)
model.fit(train,epochs=30,num_threads = 4)

In [17]:
model1 = LightFM(no_components= 30, loss='warp',k=15)
model1.fit(train,epochs=30,num_threads = 4)

In [18]:
model2 = LightFM(no_components= 30, loss='warp',k=12)
model2.fit(train,epochs=30,num_threads = 4)

In [19]:
auc_train = auc_score(model, train).mean()
auc_test = auc_score(model, test).mean()

print("Train AUC Score Model 1: {}".format(auc_train))
print("Test AUC Score Model 1: {}".format(auc_test))

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

print("Train Precision Score Model 1: {}".format(train_precision))
print("Test Precision Score Model 1: {}".format(test_precision))

Train AUC Score Model 1: 0.9597175121307373
Test AUC Score Model 1: 0.8771376013755798
Train Precision Score Model 1: 0.39367616176605225
Test Precision Score Model 1: 0.06130306422710419


In [20]:
auc_train = auc_score(model1, train).mean()
auc_test = auc_score(model1, test).mean()

print("Train AUC Score Model 2: {}".format(auc_train))
print("Test AUC Score Model 2: {}".format(auc_test))

train_precision = precision_at_k(model1, train, k=15).mean()
test_precision = precision_at_k(model1, test, k=15).mean()

print("Train Precision Score Model 2: {}".format(train_precision))
print("Test Precision Score Model 2: {}".format(test_precision))

Train AUC Score Model 2: 0.9598070979118347
Test AUC Score Model 2: 0.877265214920044
Train Precision Score Model 2: 0.3634777069091797
Test Precision Score Model 2: 0.055018100887537


In [21]:
auc_train = auc_score(model2, train).mean()
auc_test = auc_score(model2, test).mean()

print("Train AUC Score Model 3: {}".format(auc_train))
print("Test AUC Score Model 3: {}".format(auc_test))

train_precision = precision_at_k(model2, train, k=12).mean()
test_precision = precision_at_k(model2, test, k=12).mean()

print("Train Precision Score Model 3: {}".format(train_precision))
print("Test Precision Score Model 3: {}".format(test_precision))

Train AUC Score Model 3: 0.9594847559928894
Test AUC Score Model 3: 0.8776963949203491
Train Precision Score Model 3: 0.37524130940437317
Test Precision Score Model 3: 0.0582839734852314


# 1. Recommending a product by customer  <a name="recommendingproduct"></a>
> Using a customized function, we will suggest products to a customer that he/she might be interested in based on last purchased products. As an example, we made some recommendations for CustomerID = 12444.0

In [22]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = train.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
                                 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("\n Known Product Purchased:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [23]:
## Calling 10 products recommendation for CustomerID 12444
rec_list = sample_recommendation_user(model = model, 
                                      interactions = interactions1, 
                                      user_id = 12444.0, 
                                      user_dict = user_dict,
                                      item_dict = products_dict, 
                                 #     threshold =  ,
                                      nrec_items = 10,
                                      show = True)


 Known Product Purchased:
1- ZINC WILLIE WINKIE  CANDLE STICK
2- ZINC HEART LATTICE CHARGER LARGE
3- WOODEN PICTURE FRAME WHITE FINISH
4- WOODEN FRAME ANTIQUE WHITE 
5- WOODEN BOX OF DOMINOES
6- WOOD BLACK BOARD ANT WHITE FINISH
7- WIRE EGG BASKET 
8- WICKER STAR 
9- WHITE SPOT BLUE CERAMIC DRAWER KNOB
10- WHITE SOAP RACK WITH 2 BOTTLES
11- WHITE BROCANTE SOAP DISH
12- VINTAGE RED ENAMEL TRIM JUG 
13- VINTAGE COFFEE GRINDER BOX
14- VINTAGE CHRISTMAS STOCKING 
15- VICTORIAN SEWING BOX MEDIUM
16- VICTORIAN  METAL POSTCARD SPRING
17- TWO DOOR CURIO CABINET
18- STORAGE TIN HOME SWEET HOME
19- SOAP DISH BROCANTE
20- SMALL APOTHECARY MEASURING JAR 
21- SINGLE HEART ZINC T-LIGHT HOLDER
22- SHELF WITH 4 HOOKS HOME SWEET HOME
23- SET/3 VANILLA SCENTED CANDLE IN BOX
24- SET/3 DECOUPAGE STACKING TINS
25- SET OF TEA COFFEE SUGAR TINS PANTRY
26- SET OF 3 REGENCY CAKE TINS
27- SET OF 3 HEART COOKIE CUTTERS
28- SET OF 3 CAKE TINS PANTRY DESIGN 
29- SET OF 2 WOODEN MARKET CRATES
30- SET OF 2 TRAYS HO

# 2. Recommend a customer that might be interested in a specific product  <a name="recommendinguser"></a>
> Using a customized function, we will enumerate a list of customers that could be interested in a specific product. As an example, we made some recommendations for StockCode = SC90214O ('LETTER "O" BLING KEY RING')

In [24]:
def sample_recommendation_item(model,interactions,item_id,user_dict,item_dict,number_of_user):
    '''
    Funnction to produce a list of top N interested users for a given item
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - item_id = item ID for which we need to generate recommended users
        - user_dict =  Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - number_of_user = Number of users needed as an output
    Expected Output -
        - user_list = List of recommended users 
    '''
    n_users, n_items = interactions.shape
    x = np.array(interactions.columns)
    scores = pd.Series(model.predict(np.arange(n_users), np.repeat(x.searchsorted(item_id),n_users)))
    user_list = list(interactions.index[scores.sort_values(ascending=False).head(number_of_user).index])
    return user_list 

In [25]:
## Calling 15 user recommendation for item id SC90214O
sample_recommendation_item(model = model,
                           interactions = interactions1,
                           item_id = 'SC90214O',
                           user_dict = user_dict,
                           item_dict = products_dict.values,
                           number_of_user = 15)

[13672.0,
 15940.0,
 12875.0,
 16582.0,
 15945.0,
 16454.0,
 13270.0,
 13848.0,
 14046.0,
 16679.0,
 16784.0,
 18068.0,
 13391.0,
 18233.0,
 17715.0]

# Content-based Filtering

In [26]:
def create_item_emdedding_distance_matrix(model,interactions):
    '''
    Function to create item-item distance embedding matrix
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
    Expected Output -
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
    '''
    df_item_norm_sparse = sparse.csr_matrix(model.item_embeddings)
    similarities = cosine_similarity(df_item_norm_sparse)
    item_emdedding_distance_matrix = pd.DataFrame(similarities)
    item_emdedding_distance_matrix.columns = interactions.columns
    item_emdedding_distance_matrix.index = interactions.columns
    return item_emdedding_distance_matrix

def item_item_recommendation(item_emdedding_distance_matrix, item_id, 
                             item_dict, n_items = 10, show = True):
    '''
    Function to create item-item recommendation
    Required Input - 
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
        - item_id  = item ID for which we need to generate recommended items
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - n_items = Number of items needed as an output
    Expected Output -
        - recommended_items = List of recommended items
    '''
    recommended_items = list(pd.Series(item_emdedding_distance_matrix.loc[item_id,:]. \
                                  sort_values(ascending = False).head(n_items+1). \
                                  index[1:n_items+1]))
    if show == True:
        print("Item of interest :{0}".format(item_dict[item_id]))
        print("Item similar to the above item:")
        counter = 1
        for i in recommended_items:
            print(str(counter) + '- ' +  item_dict[i])
            counter+=1
    return recommended_items

In [27]:
## Creating item-item distance matrix
item_item_dist = create_item_emdedding_distance_matrix(model = model,
                                                       interactions = interactions1)
## Checking item embedding distance matrix
item_item_dist.head()

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
Description,,,,,,,,,,,,,,,,,,,,,
4 PURPLE FLOCK DINNER CANDLES,1.000000,-0.277950,-0.228397,-0.236836,0.489653,-0.036120,-0.034997,-0.239236,-0.180114,-0.077380,...,0.359039,-0.242735,-0.109498,0.107256,0.154551,-0.116636,0.243076,-0.012326,0.107158,0.084894
50'S CHRISTMAS GIFT BAG LARGE,-0.277950,1.000000,0.132208,0.136716,-0.273135,0.433925,0.024108,0.639094,-0.184597,0.326081,...,-0.368327,-0.067173,-0.024141,0.278419,0.065342,0.260526,-0.121491,0.026728,-0.196922,-0.085462
DOLLY GIRL BEAKER,-0.228397,0.132208,1.000000,0.463620,-0.061364,-0.166382,-0.151482,0.090535,0.169052,0.312438,...,-0.116579,0.007401,-0.140160,0.131221,0.073550,0.058442,-0.312496,-0.050489,-0.098830,0.076998
I LOVE LONDON MINI BACKPACK,-0.236836,0.136716,0.463620,1.000000,0.231544,-0.118206,-0.003002,0.446937,0.383366,0.425682,...,0.196424,-0.054863,-0.265870,-0.300268,0.270126,-0.386198,-0.386376,-0.461769,-0.104168,0.090480
I LOVE LONDON MINI RUCKSACK,0.489653,-0.273135,-0.061364,0.231544,1.000000,-0.081357,0.013850,0.036955,-0.117365,-0.116434,...,0.941407,-0.000180,-0.091492,-0.025137,0.661416,-0.231742,0.136638,-0.106758,0.606313,0.367301


# Recommending products that could be bought together  <a name="productsboughttogether"></a>
> Using a customized function, we will enumerate a list of products that could be bought together with specific product. As an example, we made some recommendations for Description = 'WHITE HANGING HEART T-LIGHT HOLDER'

In [28]:
## Calling 10 recommended items for item id 
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 'WHITE HANGING HEART T-LIGHT HOLDER',
                                    item_dict = products_dict,
                                    n_items = 10)

Item of interest :WHITE HANGING HEART T-LIGHT HOLDER
Item similar to the above item:
1- HEART OF WICKER LARGE
2- T-LIGHT HOLDER SWEETHEART HANGING
3- CANDLEHOLDER PINK HANGING HEART
4- RED HANGING HEART T-LIGHT HOLDER
5- HEART OF WICKER SMALL
6- PEARL CRYSTAL PUMPKIN T-LIGHT HLDR
7- BLACK HEART CARD HOLDER
8- NATURAL SLATE HEART CHALKBOARD 
9- CREAM HEART CARD HOLDER
10- HEART IVORY TRELLIS LARGE
